In [1]:
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 5.6 MB/s eta 0:00:00 MB/s eta 0:00:01


In [79]:
!pip install dask

  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.3 MB/s eta 0:00:00 MB/s eta 0:00:01
Using cached click-8.1.7-py3-none-any.whl (97 kB)


In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
import dask.dataframe as dd
from rapidfuzz import process, fuzz
from datetime import datetime

/opt/miniconda3/envs/ML_Course/lib/python3.12/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
fuzz.WRatio("this is a test", "this is a new test!!!")

85.5

In [85]:
fuzz.token_set_ratio("this is a test", "this is a new test!!!")

80.0

In [94]:
fuzz.WRatio("Love Song Taylor A", "Love Song (Taylor)")

88.88888888888889

In [93]:
fuzz.token_set_ratio("Love Song Taylor AA", "Love Song (Taylor)")

86.48648648648648

In [175]:
fuzz.WRatio("Out of This World", "Always Running Out of Time")

85.5

In [178]:
from rapidfuzz.distance import Levenshtein
Levenshtein.normalized_similarity("Out of This World", "Always Running Out of Time")

0.15384615384615385

In [179]:
from rapidfuzz.distance import Levenshtein
Levenshtein.normalized_similarity("Circus Music", "Exit Music (For A Dub)")

0.18181818181818177

### Load data

In [2]:
# # Load only necessary columns
# df_opendb = pd.read_csv('/Users/anhnhat/Library/Mobile Documents/com~apple~CloudDocs/Documents/UNIMA/2. Semester Study/2. WS2425/4. W24 Web Data Integration/Project/Reduce Dataset/opendb_s.csv', usecols=['track_title'])

In [3]:
# df_opendb.head(5)

In [4]:
# # Load only necessary columns
# df_am = pd.read_csv('/Users/anhnhat/Library/Mobile Documents/com~apple~CloudDocs/Documents/UNIMA/2. Semester Study/2. WS2425/4. W24 Web Data Integration/Project/Reduce Dataset/apple_music_dataset.csv',usecols=['trackCensoredName'])

In [5]:
# # Load only necessary columns
# df_am = pd.read_csv('/Users/anhnhat/Library/Mobile Documents/com~apple~CloudDocs/Documents/UNIMA/2. Semester Study/2. WS2425/4. W24 Web Data Integration/Project/Reduce Dataset/apple_music_dataset.csv')

In [6]:
# df_am.head()

In [7]:
path = '/Users/anhnhat/Library/Mobile Documents/com~apple~CloudDocs/Documents/UNIMA/2. Semester Study/2. WS2425/4. W24 Web Data Integration/Project/Reduce Dataset/'

In [8]:
# Apple music
ids, tracks = [], []

# Parse XML file iteratively
for event, elem in ET.iterparse(path + 'apple.xml', events=('end',)):
    if elem.tag == 'song':  # Look for each <song> element
        # Extract ID and Track fields
        id_value = elem.find('id').text if elem.find('id') is not None else None
        track_value = elem.find('Track').text if elem.find('Track') is not None else None
        
        # Append extracted data to lists
        ids.append(id_value)
        tracks.append(track_value)

        # Clear element to free memory
        elem.clear()

# Create DataFrame with the extracted columns
df_am = pd.DataFrame({'ID': ids, 'Track': tracks})

# Display the DataFrame to verify
print(df_am.head())


   ID                                 Track
0  A1                                 Music
1  A2                                 Music
2  A3  Don't Stop Believin' (2024 Remaster)
3  A4                             I'm Yours
4  A5                                 Music


In [9]:
def extract_id_and_track(xml_file):
    # Initialize lists to store the extracted data
    ids, tracks = [], []

    # Parse the XML file iteratively
    for event, elem in ET.iterparse(path + xml_file, events=('end',)):
        if elem.tag == 'song':  # Look for each <song> element
            # Extract ID and Track fields
            id_value = elem.find('id').text if elem.find('id') is not None else None
            track_value = elem.find('Track').text if elem.find('Track') is not None else None
            
            # Append extracted data to lists
            ids.append(id_value)
            tracks.append(track_value)

            # Clear element to free memory
            elem.clear()

    # Create DataFrame with the extracted columns
    df = pd.DataFrame({'ID': ids, 'Track': tracks})

    # Display the DataFrame to verify
    return df


In [10]:
# Apple Music
df_am = extract_id_and_track('apple.xml')
print(df_am.head())

   ID                                 Track
0  A1                                 Music
1  A2                                 Music
2  A3  Don't Stop Believin' (2024 Remaster)
3  A4                             I'm Yours
4  A5                                 Music


In [11]:
# Example usage:
df_million = extract_id_and_track('million.xml')
print(df_million.head())

   ID            Track
0  C1   Mr. Brightside
1  C2       Wonderwall
2  C3  Come as You Are
3  C4      Take Me Out
4  C5            Creep


In [12]:
# Example usage:
df_opendb= extract_id_and_track('opendb_s.xml')
print(df_opendb.head())

   ID                                            Track
0  B1                                 Little Girl Blue
1  B2                    Summertime blues (En directo)
2  B3                 Man In Meridian (bossman Birdie)
3  B4                           Better Man (feat. Reo)
4  B5  You're All I Need To Get By (Glee Cast Version)


In [13]:
print('Number rows of dataset Apple Music: ' + str(df_am.shape[0]))
print('Number rows of dataset Millions: ' + str(df_million.shape[0]))
print('Number rows of dataset Open DB: ' + str(df_opendb.shape[0]))

Number rows of dataset Apple Music: 10000
Number rows of dataset Millions: 50683
Number rows of dataset Open DB: 100001


In [187]:
df_opendb_100k = df_opendb.sample(n=100000)

In [188]:
print('Number rows of dataset Open DB random 100k: ' + str(df_opendb_100k.shape[0])) 

Number rows of dataset Open DB random 100k: 100000


In [162]:
# df_am = df_am.sample(n=2000)
# df_million = df_million.sample(n=2000)
# df_opendb_100k = df_opendb_100k.sample(n=10000)

In [164]:
from datetime import datetime
datetime.now().strftime('%Y-%m-%d %H:%M:%S')

'2024-10-30 23:55:49'

In [165]:
def compare_large_with_small(df_large, df_small, large_track_col='Track', small_track_col='Track', npartitions=10):
    """
    Compare a large Dask DataFrame with a smaller Pandas DataFrame using RapidFuzz similarity metrics.
    
    Parameters:
        df_large (pd.DataFrame or dd.DataFrame): Large Dask DataFrame to compare.
        df_small (pd.DataFrame): Smaller Pandas DataFrame for comparison.
        large_track_col (str): Column name in df_large containing track names.
        small_track_col (str): Column name in df_small containing track names.
        npartitions (int): Number of partitions for Dask DataFrame.
        
    Returns:
        dd.DataFrame: Dask DataFrame with similarity scores and matched track from df_small.
    """
    # Convert large DataFrame to Dask if it's not already
    if not isinstance(df_large, dd.DataFrame):
        df_large = dd.from_pandas(df_large, npartitions=npartitions)

    # Define the function to get the similarity score and matched track
    def get_match_details(track):
        best_match = process.extractOne(track, df_small[small_track_col], scorer=fuzz.WRatio)
        score, matched_track = best_match[1], best_match[0] if best_match else (None, None)
        return score, matched_track

    # Apply the function and separate the results into two columns
    df_large['match_details'] = df_large[large_track_col].apply(get_match_details, meta=(large_track_col, 'object'))
    df_large = df_large.assign(
        similarity_score=df_large['match_details'].apply(lambda x: x[0], meta=('x', 'float')),
        matched_track=df_large['match_details'].apply(lambda x: x[1], meta=('x', 'object'))
    )

    # Drop the intermediate tuple column
    df_large = df_large.drop(columns='match_details')

    return df_large

# Example usage
# Compare large dataset with df_am
df_result_am = compare_large_with_small(df_opendb_100k, df_am)
df_result_am = df_result_am.compute()  # Convert back to Pandas if needed

# Compare large dataset with million
df_result_million = compare_large_with_small(df_opendb_100k,df_million )
df_result_million = df_result_million.compute()  # Convert back to Pandas if needed


In [168]:
df_result_am.head()

,ID,Track,similarity_score,matched_track
66,B67,Replay (O meu time é a alegria da cidade),90.0,Replay
98,B99,Replay,100.0,Replay
287,B288,Replay,100.0,Replay
411,B412,Replay (Sped Up Version) (feat. Ashley Jana),90.0,Replay
606,B607,Circus Music,90.0,Music


In [169]:
df_result_million.head()

,ID,Track,similarity_score,matched_track
66,B67,Replay (O meu time é a alegria da cidade),85.500000,Ghost of a Chance
98,B99,Replay,65.454545,Ready 2 Wear
287,B288,Replay,65.454545,Ready 2 Wear
411,B412,Replay (Sped Up Version) (feat. Ashley Jana),85.500000,Further On Up The Road
606,B607,Circus Music,85.500000,Exit Music (For A Dub)


In [170]:
# Ensure the results are in Pandas format
df_result_am = df_result_am.compute() if isinstance(df_result_am, dd.DataFrame) else df_result_am
df_result_million = df_result_million.compute() if isinstance(df_result_million, dd.DataFrame) else df_result_million

# Write each DataFrame to CSV
df_result_am.to_csv('df_result_am.csv', index=False)
df_result_million.to_csv('df_result_million.csv', index=False)


In [180]:
from datetime import datetime
datetime.now().strftime('%Y-%m-%d %H:%M:%S')


'2024-10-31 00:20:33'

### normalized Levenshtein similarity

In [17]:
import pandas as pd
import dask.dataframe as dd
from rapidfuzz import process
from rapidfuzz.distance import Levenshtein
from datetime import datetime

def compare_large_with_small(df_large, df_small, output_file, large_track_col='Track', small_track_col='Track', npartitions=10, score_threshold=0.6):
    """
    Compare a large Dask DataFrame with a smaller Pandas DataFrame using Levenshtein normalized similarity with a score threshold,
    track runtime, and save results to a CSV file.
    
    Parameters:
        df_large (pd.DataFrame or dd.DataFrame): Large Dask DataFrame to compare.
        df_small (pd.DataFrame): Smaller Pandas DataFrame for comparison.
        output_file (str): Name of the output CSV file.
        large_track_col (str): Column name in df_large containing track names.
        small_track_col (str): Column name in df_small containing track names.
        npartitions (int): Number of partitions for Dask DataFrame.
        score_threshold (float): Minimum similarity score to consider a match, between 0 and 1.
        
    Returns:
        dd.DataFrame: Dask DataFrame with similarity scores, matched track, and matched ID from df_small.
    """
    # Record start time
    start_time = datetime.now()
    print(f"Start Time: {start_time}")

    # Convert large DataFrame to Dask if it's not already
    if not isinstance(df_large, dd.DataFrame):
        df_large = dd.from_pandas(df_large, npartitions=npartitions)

    # Define the function to get the normalized Levenshtein similarity score, matched track, and matched ID
    def get_match_details(track):
        best_match = process.extractOne(
            track,
            df_small[small_track_col],
            scorer=Levenshtein.normalized_similarity
            #score_cutoff=score_threshold  # Apply the score threshold
        )
        if best_match:
            score, matched_track = best_match[1], best_match[0]
            matched_id = df_small[df_small[small_track_col] == matched_track]["ID"].values[0]
        else:
            score, matched_track, matched_id = None, None, None
        return score, matched_track, matched_id

    # Apply the function and separate the results into three columns
    df_large['match_details'] = df_large[large_track_col].apply(get_match_details, meta=(large_track_col, 'object'))
    df_large = df_large.assign(
        similarity_score=df_large['match_details'].apply(lambda x: x[0], meta=('x', 'float')),
        matched_track=df_large['match_details'].apply(lambda x: x[1], meta=('x', 'object')),
        matched_id=df_large['match_details'].apply(lambda x: x[2], meta=('x', 'object'))
    )

    # Drop the intermediate tuple column
    df_large = df_large.drop(columns='match_details')

    # Compute the Dask DataFrame to get the result in Pandas format
    df_result = df_large.compute()

    # Record end time
    end_time = datetime.now()
    print(f"End Time: {end_time}")

    # Calculate and print runtime
    runtime = end_time - start_time
    print(f"Runtime: {runtime}")

    # Write the result to CSV
    df_result.to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")

    return df_result

# Example usage
# Compare large dataset with df_am and save results to CSV with score threshold of 0.6
#df_result_am = compare_large_with_small(df_opendb, df_am, 'df_result_am.csv', score_threshold=0.6)
df_result_am = compare_large_with_small(df_opendb, df_am, 'df_result_am.csv')

# Compare large dataset with df_million and save results to CSV with score threshold of 0.6
#df_result_million = compare_large_with_small(df_opendb, df_million, 'df_result_million.csv', score_threshold=0.6)
df_result_million = compare_large_with_small(df_opendb, df_million, 'df_result_million.csv')


Start Time: 2024-10-31 16:39:21.887359
End Time: 2024-10-31 16:49:47.338898
Runtime: 0:10:25.451539
Results saved to df_result_am.csv
Start Time: 2024-10-31 16:49:47.482583
End Time: 2024-10-31 17:15:37.998957
Runtime: 0:25:50.516374
Results saved to df_result_million.csv


In [ ]:
import pandas as pd
import dask.dataframe as dd
from rapidfuzz import process
from rapidfuzz.distance import Levenshtein
from datetime import datetime

def compare_large_with_small_noscore(df_large, df_small, output_file, large_track_col='Track', small_track_col='Track', large_id_col='ID', small_id_col='ID', npartitions=10, score_threshold=0.6):
    """
    Compare a large Dask DataFrame with a smaller Pandas DataFrame using Levenshtein normalized similarity with a score threshold,
    track runtime, and save results to a CSV file, including IDs from both DataFrames.
    
    Parameters:
        df_large (pd.DataFrame or dd.DataFrame): Large Dask DataFrame to compare.
        df_small (pd.DataFrame): Smaller Pandas DataFrame for comparison.
        output_file (str): Name of the output CSV file.
        large_track_col (str): Column name in df_large containing track names.
        small_track_col (str): Column name in df_small containing track names.
        large_id_col (str): Column name in df_large containing IDs.
        small_id_col (str): Column name in df_small containing IDs.
        npartitions (int): Number of partitions for Dask DataFrame.
        score_threshold (float): Minimum similarity score to consider a match, between 0 and 1.
        
    Returns:
        dd.DataFrame: Dask DataFrame with similarity scores and matched track from df_small.
    """
    # Record start time
    start_time = datetime.now()
    print(f"Start Time: {start_time}")

    # Convert large DataFrame to Dask if it's not already
    if not isinstance(df_large, dd.DataFrame):
        df_large = dd.from_pandas(df_large, npartitions=npartitions)

    # Define the function to get the normalized Levenshtein similarity score, matched track, and matched ID
    def get_match_details(row):
        track = row[large_track_col]
        best_match = process.extractOne(
            track,
            df_small[[small_track_col, small_id_col]].values.tolist(),
            scorer=Levenshtein.normalized_similarity,
            processor=lambda x: x[0]  # Extract the track name for similarity matching
        )
        if best_match:
            score = best_match[1]
            matched_track, matched_id = best_match[0]
        else:
            score, matched_track, matched_id = None, None, None
        return score, matched_track, matched_id

    # Apply the function and separate the results into multiple columns
    df_large['match_details'] = df_large.apply(get_match_details, axis=1, meta=(large_track_col, 'object'))
    df_large = df_large.assign(
        similarity_score=df_large['match_details'].apply(lambda x: x[0], meta=('x', 'float')),
        matched_track=df_large['match_details'].apply(lambda x: x[1], meta=('x', 'object')),
        matched_id=df_large['match_details'].apply(lambda x: x[2], meta=('x', 'object'))
    )

    # Drop the intermediate tuple column and include original ID from the large DataFrame
    df_large = df_large.drop(columns='match_details')
    df_large['large_id'] = df_large[large_id_col]

    # Compute the Dask DataFrame to get the result in Pandas format
    df_result = df_large.compute()

    # Record end time
    end_time = datetime.now()
    print(f"End Time: {end_time}")

    # Calculate and print runtime
    runtime = end_time - start_time
    print(f"Runtime: {runtime}")

    # Write the result to CSV
    df_result.to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")

    return df_result

# Example usage
# Compare large dataset with df_am and save results to CSV with score threshold of 0.6
#df_result_am = compare_large_with_small(df_opendb, df_am, 'df_result_am.csv', score_threshold=0.6)
df_result_am = compare_large_with_small_noscore(df_opendb, df_am, 'df_result_am.csv')

# Compare large dataset with df_million and save results to CSV with score threshold of 0.6
df_result_million = compare_large_with_small_noscore(df_opendb, df_million, 'df_result_million.csv')

Start Time: 2024-10-31 16:09:59.123551
End Time: 2024-10-31 16:14:31.133282
Runtime: 0:04:32.009731
Results saved to df_result_am.csv
Start Time: 2024-10-31 16:14:31.272288


KeyboardInterrupt: 

In [24]:
from rapidfuzz.distance import Levenshtein

# Define the two strings
string1 = "Little Girl Blue"
string2 = "L"

# Calculate the Levenshtein distance and normalized similarity
distance = Levenshtein.distance(string1, string2)
normalized_similarity = Levenshtein.normalized_similarity(string1, string2)

print("Levenshtein Distance:", distance)
print("Normalized Similarity:", normalized_similarity)


Levenshtein Distance: 15
Normalized Similarity: 0.0625
